### Nosso Objetivo: 


Criar um modelo de Machine Learning que aprenda com as vagas existentes para prever o salario_avg de uma nova vaga que nunca vimos antes.

In [73]:
import pandas as pd
import sqlite3

# --- ETAPA DE CARREGAMENTO E LIMPEZA INICIAL ---

# Configuração
db_file_path = 'data_science_jobs.db'
table_name = 'job_postings'

# Carregando os Dados
conn = sqlite3.connect(db_file_path)
query = f"SELECT * FROM {table_name}"
df = pd.read_sql_query(query, conn)
conn.close()

print(f"Dados carregados. Shape inicial: {df.shape}")

# --- LIMPEZA E VALIDAÇÃO ---

# 1. (BOA PRÁTICA) Remover linhas onde o alvo (salario_avg) é nulo
df.dropna(subset=['salario_avg'], inplace=True)
print(f"Shape após remover salários nulos: {df.shape}")

# 2. (A SOLUÇÃO FINAL) Remover outliers extremos de salário
df = df[df['salario_avg'] < 2000000] # Mantém apenas as linhas com salário ABAIXO de 2 milhões
print(f"Shape após remover outliers: {df.shape}")


# Exibe as 5 primeiras linhas do DataFrame limpo para verificação
print("\n--- Amostra dos Dados Limpos ---")
display(df.head())

Dados carregados. Shape inicial: (944, 35)
Shape após remover salários nulos: (944, 35)
Shape após remover outliers: (942, 35)

--- Amostra dos Dados Limpos ---


,job_title,seniority_level,status,company,location,post_date,headquarter,industry,ownership,company_size,...,tem_scikit_learning,tem_tensorflow,tem_pytorch,tem_azure,tem_gcp,tem_tableau,tem_pandas,tem_git,tem_java,tem_powerbi
0,Cientista de Dados,Sênior,Híbrido,company_003,"Grapevine, TX . Hybrid",17 days ago,"Bentonville, AR, US",Varejo,Pública,€352.44B,...,0,1,0,0,0,0,0,0,0,0
1,Cientista de Dados,Líder,Híbrido,company_005,"Fort Worth, TX . Hybrid",15 days ago,"Detroit, MI, US",Manufatura,Pública,"155,030",...,0,0,0,0,0,0,0,0,0,0
2,Cientista de Dados,Sênior,Presencial,company_007,"Austin, TX . Toronto, Ontario, Canada . Kirkla...",a month ago,"Redwood City, CA, US",Tecnologia,Pública,"25,930",...,0,0,0,0,1,0,0,1,0,0
3,Cientista de Dados,Sênior,Híbrido,company_008,"Chicago, IL . Scottsdale, AZ . Austin, TX . Hy...",8 days ago,"San Jose, CA, US",Tecnologia,Pública,"34,690",...,0,0,0,0,0,0,0,0,0,0
4,Cientista de Dados,Não Informado,Presencial,company_009,On-site,3 days ago,"Stamford, CT, US",Finanças,Privada,"1,800",...,0,0,0,0,0,0,0,0,0,0


In [74]:
# Coloque esta célula logo após o carregamento inicial do df

print("--- Verificação Geral do DataFrame (df.info()) ---")
df.info()

print("\n\n--- Resumo Estatístico Completo do DataFrame (df.describe()) ---")
# O include='all' mostra estatísticas para colunas de texto e numéricas
display(df.describe(include='all'))

--- Verificação Geral do DataFrame (df.info()) ---
<class 'pandas.core.frame.DataFrame'>
Index: 942 entries, 0 to 943
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   job_title             942 non-null    object 
 1   seniority_level       942 non-null    object 
 2   status                942 non-null    object 
 3   company               942 non-null    object 
 4   location              940 non-null    object 
 5   post_date             942 non-null    object 
 6   headquarter           942 non-null    object 
 7   industry              942 non-null    object 
 8   ownership             895 non-null    object 
 9   company_size          942 non-null    object 
 10  revenue               927 non-null    object 
 11  salary                942 non-null    object 
 12  skills                942 non-null    object 
 13  salario_min           942 non-null    float64
 14  salario_max           942 no

,job_title,seniority_level,status,company,location,post_date,headquarter,industry,ownership,company_size,...,tem_scikit_learning,tem_tensorflow,tem_pytorch,tem_azure,tem_gcp,tem_tableau,tem_pandas,tem_git,tem_java,tem_powerbi
count,942,942,942,942,940,942,942,942,895,942,...,942.000000,942.000000,942.000000,942.000000,942.000000,942.000000,942.000000,942.000000,942.000000,942.000000
unique,5,5,4,420,429,42,197,8,2,510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Cientista de Dados,Sênior,Presencial,company_134,"Bengaluru, Karnataka, India",a month ago,"San Francisco, CA, US",Tecnologia,Pública,900,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,854,628,363,30,52,167,91,580,579,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.096603,0.175159,0.157113,0.162420,0.111465,0.123142,0.080679,0.069002,0.077495,0.026539
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.295573,0.380305,0.364100,0.369032,0.314874,0.328775,0.272487,0.253592,0.267517,0.160818
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [75]:
# Vamos encontrar a(s) vaga(s) com salário astronômico
outliers = df[df['salario_avg'] > 2000000] # Filtra o DataFrame para salários maiores que 2 milhões

print("--- Vaga(s) com Salário Extremo Encontrada(s) ---")
display(outliers)

--- Vaga(s) com Salário Extremo Encontrada(s) ---


,job_title,seniority_level,status,company,location,post_date,headquarter,industry,ownership,company_size,...,tem_scikit_learning,tem_tensorflow,tem_pytorch,tem_azure,tem_gcp,tem_tableau,tem_pandas,tem_git,tem_java,tem_powerbi


### Vamos dividir nossa construção do modelo em etapas claras:

- Etapa 1: Seleção e Preparação das Features (Onde estamos agora).

- Etapa 2: Pré-processamento e Codificação (Transformando tudo em números).

- Etapa 3: Divisão dos Dados (Treino e Teste).

- Etapa 4: Escolha e Treinamento do Modelo.

- Etapa 5: Avaliação do Modelo.

## Etapa 1: Selecionando as Armas (Features) e o Alvo (Target)

1. O Alvo (Target): O que você quer que eu aprenda a prever? ( y )

2. As Pistas (Features): Que informações você vai me dar para que eu possa fazer a previsão? ( x )

### O nosso Alvo (y): 
É fácil. Queremos prever o salário. A coluna perfeita para isso é a salario_avg que nós criamos.


### As nossas Pistas (X): 
Aqui entra a parte estratégica. Quais colunas do nosso banco de dados são pistas úteis para adivinhar o salário?

- job_title, seniority_level, status, industry: Com certeza! São muito importantes.
- pais, estado: Sim, o local influencia no salário.
- tem_python, tem_sql, etc.: Absolutamente! As habilidades são um fator chave.

E quais colunas devemos EXCLUIR das pistas?

- salary, salario_min, salario_max, salario_avg: NUNCA podemos usar o próprio alvo ou suas variações como uma pista. Seria como dar a resposta da prova para o aluno. Isso se chama data leakage (vazamento de dados) e é o erro número 1 em projetos de ML.
- location, skills: As colunas de texto originais. Elas são "sujas" e nós já extraímos as informações importantes delas para as novas colunas.
- company, post_date, headquarter: São identificadores, datas em formato de texto ou informações que já foram representadas de forma melhor (pais, estado). Por agora, vamos deixá-las de fora para manter nosso primeiro modelo simples e forte.

In [76]:
# ---  Etapa 1: Seleção de Features e target  ---

# 1. Definir o Alvo (Target)
# A variável 'y' vai conter apenas a coluna que queremos prever.
target = 'salario_avg'
y = df[target]

# 2. Definir as Pistas (Features)
# Criamos uma lista com os nomes de todas as colunas que servirão como "pistas" para o modelo.
features = [
    'job_title', 'seniority_level', 'status', 'industry', 'ownership',
    'pais', 'estado',
    'tem_python', 'tem_sql', 'tem_r', 'tem_machine_learning', 'tem_aws',
    'tem_spark', 'tem_deep_learning', 'tem_scikit_learning', 'tem_tensorflow',
    'tem_pytorch', 'tem_azure', 'tem_gcp', 'tem_tableau', 'tem_pandas',
    'tem_git', 'tem_java', 'tem_powerbi'
]
X = df[features]

# Tratamento de valores nulos ANTES do encoding
X.fillna("Não Informado", inplace=True)

# 3. Verificar o resultado
print("--- Amostra do Alvo (y) ---")
print(y.head())
print("\n--- Amostra das Features (X) ---")
print(X.head())

print(f"\nFormato de X (linhas, colunas): {X.shape}")
print(f"Formato de y (linhas,): {y.shape}")

--- Amostra do Alvo (y) ---
0    150705.0
1    118733.0
2    127273.0
3    153599.5
4    171254.5
Name: salario_avg, dtype: float64

--- Amostra das Features (X) ---
            job_title seniority_level      status    industry ownership  \
0  Cientista de Dados          Sênior     Híbrido      Varejo   Pública   
1  Cientista de Dados           Líder     Híbrido  Manufatura   Pública   
2  Cientista de Dados          Sênior  Presencial  Tecnologia   Pública   
3  Cientista de Dados          Sênior     Híbrido  Tecnologia   Pública   
4  Cientista de Dados   Não Informado  Presencial    Finanças   Privada   

            pais         estado  tem_python  tem_sql  tem_r  ...  \
0             US             TX           1        0      1  ...   
1             US             TX           1        1      1  ...   
2             CA  Não Informado           1        1      0  ...   
3             US             IL           1        1      1  ...   
4  Não Informado  Não Informado           0

C:\Users\Guilh\AppData\Local\Temp\ipykernel_11672\3174086559.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna("Não Informado", inplace=True)


## Etapa 2: Pré-processamento e Codificação (One-Hot Encoding)

O Problema: Temos colunas como job_title com valores 'Cientista de Dados', 'Engenheiro de Dados', etc. Não podemos simplesmente substituir 'Cientista de Dados' por 1 e 'Engenheiro de Dados' por 2, pois isso criaria uma relação matemática falsa (como se um Engenheiro fosse "o dobro" de um Cientista), o que confundiria o modelo.

A Solução: One-Hot Encoding
A técnica correta se chama One-Hot Encoding. É um nome complicado para uma ideia muito simples, que eu chamo de "Técnica do Interruptor".

Imagine a coluna status com 3 opções: 'Presencial', 'Híbrido', 'Remoto'.
O One-Hot Encoding vai fazer o seguinte:

1. Ele apaga a coluna status original.

2. Ele cria novas colunas, uma para cada opção possível, como se fossem interruptores: status_Presencial, status_Híbrido, status_Remoto.

3. Para cada vaga, ele "liga" (1) apenas o interruptor correspondente e deixa 
os outros "desligados" (0).


Isso transforma uma única coluna de texto em múltiplas colunas de 0s e 1s, sem criar relações falsas. O modelo agora entende perfeitamente a categoria de cada vaga.

In [77]:
# --- Etapa 2: Pré-processamento e Codificação (One-Hot Encoding) ---

# O pandas tem uma função "mágica" para isso chamada get_dummies().
# Ela encontra todas as colunas que não são numéricas no DataFrame X e aplica a "Técnica do Interruptor".
# O parâmetro drop_first=True é uma boa prática para evitar redundância de dados, o que pode ajudar o modelo.
X_encoded = pd.get_dummies(X, drop_first=True).astype(int)

# --- Verificação ---
print("--- Amostra das Features após o One-Hot Encoding ---")
print(X_encoded.head())

print("\n--- Mudança no Formato do DataFrame ---")
print(f"Formato original de X (antes do encoding): {X.shape}")
print(f"Formato de X_encoded (após o encoding): {X_encoded.shape}")

--- Amostra das Features após o One-Hot Encoding ---
   tem_python  tem_sql  tem_r  tem_machine_learning  tem_aws  tem_spark  \
0           1        0      1                     1        0          1   
1           1        1      1                     1        0          1   
2           1        1      0                     1        1          0   
3           1        1      1                     0        0          0   
4           0        0      0                     0        0          0   

   tem_deep_learning  tem_scikit_learning  tem_tensorflow  tem_pytorch  ...  \
0                  0                    0               1            0  ...   
1                  0                    0               0            0  ...   
2                  1                    0               0            0  ...   
3                  0                    0               0            0  ...   
4                  0                    0               0            0  ...   

   estado_ON  estado_

## Etapa 3: Divisão dos Dados (Treino e Teste)

Como falamos antes, precisamos separar nossos dados. Vamos usar a maior parte deles (geralmente 80%) para treinar o modelo. Os 20% restantes vamos guardar em um "cofre" e só usar no final, para fazer uma "prova final" e ver se o modelo é bom com dados que ele nunca viu antes.

A biblioteca scikit-learn, que é o padrão da indústria para ML em Python, tem uma função perfeita para isso: a train_test_split.

In [78]:
# --- Etapa 3: Divisão dos Dados em Treino e Teste ---
from sklearn.model_selection import train_test_split

# Esta função vai embaralhar e dividir nossos dados em 4 partes:

# X_train, y_train: 80% dos dados para o modelo aprender.
# X_test, y_test: 20% dos dados para testarmos o modelo no final.
# random_state=42: É como uma "semente" para o embaralhamento. Usar um número fixo grande que a divisão seja sempre a mesma, tornando nosso experimento reprodutível.

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# --- Verificação ---
print("--- Formato dos Dados de Treino ---")
print(f"Formato de X_train: {X_train.shape}")
print(f"Formato de y_train: {y_train.shape}")

print("\n--- Formato dos Dados de Teste ---")
print(f"Formato de X_test: {X_test.shape}")
print(f"Formato de y_test: {y_test.shape}")

--- Formato dos Dados de Treino ---
Formato de X_train: (753, 86)
Formato de y_train: (753,)

--- Formato dos Dados de Teste ---
Formato de X_test: (189, 86)
Formato de y_test: (189,)


O "Desempacotamento" Mágico do Python
A função train_test_split não retorna 4 variáveis soltas. Na verdade, ela retorna uma única coisa: uma estrutura de dados chamada tupla, que contém 4 elementos dentro dela, sempre na mesma ordem.

A documentação da scikit-learn garante que a ordem de retorno é sempre:

1. O conjunto de treino das features (X_train)

2. O conjunto de teste das features (X_test)

3. O conjunto de treino do alvo (y_train)

4. O conjunto de teste do alvo (y_test)

Ordem que escrevemos X_train, X_test, y_train, y_test é fixa e obrigatória.

## Etapa 4: Escolhendo o Cérebro (O Algoritmo) e Treinando o Modelo

Existem dezenas de algoritmos, cada um com uma forma diferente de "pensar". Para o nosso primeiro projeto, vamos começar com um dos mais fundamentais e importantes: a Regressão Linear (LinearRegression).


O que é a Regressão Linear?
Pense nela como um detetive que tenta encontrar a "fórmula" ou a "receita" do salário. Ele vai olhar para todas as nossas features (tem_python, seniority_level_Sênior, pais_US, etc.) e atribuir um "peso" para cada uma.

- Ele pode aprender que seniority_level_Sênior tem um peso positivo alto (aumenta o salário).

- Pode aprender que tem_python tem um peso positivo moderado.

- Pode aprender que job_title_Analista de Dados tem um peso negativo (diminui o salário em relação à média).

O "treinamento" do modelo é o processo de encontrar os valores exatos para todos esses pesos.

In [79]:
# --- CÉLULA DE DIAGNÓSTICO DE NÍVEL PROFUNDO ---
import numpy as np

print("--- Inspeção Profunda dos Dados de Treino ---")

# 1. Verificar o tipo de dado do array numérico que o modelo realmente recebe
print(f"Tipo de dado de X_train (array interno): {X_train.values.dtype}")
print(f"Tipo de dado de y_train (array interno): {y_train.values.dtype}")

# 2. A verificação definitiva: TODOS os valores são finitos (não-nulos E não-infinitos)?
print(f"Todos os valores em X_train são finitos? {np.isfinite(X_train.values).all()}")
print(f"Todos os valores em y_train são finitos? {np.isfinite(y_train.values).all()}")
print("-------------------------------------------------------------------------")

--- Inspeção Profunda dos Dados de Treino ---
Tipo de dado de X_train (array interno): int32
Tipo de dado de y_train (array interno): float64
Todos os valores em X_train são finitos? True
Todos os valores em y_train são finitos? True
-------------------------------------------------------------------------


In [80]:
# --- Etapa 4: Escolha e Treinamento do Modelo (VERSÃO COM RANDOM FOREST) ---
from sklearn.ensemble import RandomForestRegressor

# 1. Criar o modelo
# Trocamos o cérebro! Em vez de LinearRegression, usamos RandomForestRegressor.
# random_state=42 garante que a "floresta" seja sempre criada da mesma forma, para reprodutibilidade.
model = RandomForestRegressor(random_state=42)

# 2. Treinar o modelo
# A MÁGICA: O comando .fit() é EXATAMENTE O MESMO!
# O scikit-learn tem uma API consistente, não importa o quão complexo seja o modelo.
model.fit(X_train, y_train)

# 3. Verificação
print("Modelo RandomForest treinado com sucesso! 🌳🌳🌳")

Modelo RandomForest treinado com sucesso! 🌳🌳🌳


## Etapa 5: A Prova Final (Avaliando o Desempenho do Modelo)

No entanto, para um modelo de Regressão (que prevê um número, como o salário), não usamos "porcentagem de acerto", porque seria impossível o modelo acertar o valor exato sempre (prever €145.231,78 em vez de €145.231,80 já seria um "erro").

Em vez disso, usamos outras métricas para saber se ele está "chegando perto". As duas mais importantes para nós serão:

### 1. MAE (Mean Absolute Error - Erro Médio Absoluto):

- O que é? Em média, quantos euros o nosso modelo erra a previsão (para mais ou para menos).

- Como ler? Se o MAE for €10.000, significa que, na média, as previsões do nosso modelo estão a €10.000 de distância do salário real. Um MAE menor é melhor.

### 2. R² (R-quadrado ou Coeficiente de Determinação):

- O que é? Essa é a métrica que mais se aproxima da sua ideia de "porcentagem". Ela nos diz que porcentagem da variação dos salários o nosso modelo consegue explicar com base nas features que demos a ele.

- Como ler? O valor vai de 0 a 1 (ou 0% a 100%). Um R² de 0.65, por exemplo, significa que nosso modelo consegue explicar 65% dos motivos que fazem um salário ser alto ou baixo. O resto (35%) se deve a fatores que não estão nos nossos dados (sorte, habilidade de negociação, etc.). Um R² maior é melhor. Para um problema complexo como salários, um R² acima de 0.60 ou 0.70 já é considerado um bom resultado!

In [81]:
# --- Etapa 5: Avaliação do Modelo ---
from sklearn.metrics import mean_absolute_error, r2_score

# 1. Fazer as previsões no conjunto de teste
# Usamos o comando .predict() nos dados de teste (X_test) para ver o que o modelo "adivinha".
y_pred = model.predict(X_test)

# 2. Calcular as métricas
# Comparamos as previsões (y_pred) com as respostas reais (y_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 3. Exibir os resultados
print("--- Avaliação do Modelo RandomForest ---")
print(f"Erro Médio Absoluto (MAE): € {mae:,.2f}")
print(f"Coeficiente de Determinação (R²): {r2:.2%}")

--- Avaliação do Modelo RandomForest ---
Erro Médio Absoluto (MAE): € 29,514.79
Coeficiente de Determinação (R²): 60.07%


## Etapa 6: Otimizando o Modelo (Ajuste de Hiperparâmetros)

### o Ajuste de Hiperparâmetros.

Vou usar uma analogia com um aluno estudando para uma prova:

- Treinamento do Modelo (o que já fizemos): Foi como o aluno (nosso modelo) estudando o livro (os dados X_train) uma única vez, do começo ao fim, para aprender para a prova (y_train).

- Ajuste de Hiperparâmetros (o que vamos fazer): Agora, nós seremos o professor tentando descobrir a melhor ESTRATÉGIA de estudo para o aluno. O aluno ainda vai estudar o livro todo de uma vez, mas vamos testar diferentes abordagens para ver qual gera o melhor resultado. Por exemplo:

   - Estratégia 1: Estude por 1 hora (n_estimators=100) e foque apenas nos conceitos gerais (max_depth=10).

   - Estratégia 2: Estude por 3 horas (n_estimators=300) e foque apenas nos conceitos gerais (max_depth=10).

   - Estratégia 3: Estude por 1 hora (n_estimators=100), mas agora se aprofunde muito nos detalhes (max_depth=20).

Então, não vamos treinar o mesmo modelo 10 vezes para ele "lembrar" mais. Vamos treinar 10 modelos ligeiramente diferentes (com estratégias de estudo/hiperparâmetros diferentes) e, no final, escolher o melhor deles.

In [82]:
# --- Etapa 6: Otimização do Modelo com GridSearchCV ---
from sklearn.model_selection import GridSearchCV

# 1. Definir a grade de hiperparâmetros que queremos testar
# Estes são os "botões" do nosso RandomForest que vamos ajustar.
param_grid = {
    'n_estimators': [100, 200],     # Número de árvores na floresta
    'max_depth': [10, 20, None],    # Profundidade máxima de cada árvore
    'min_samples_split': [2, 5]     # Número mínimo de amostras para dividir um nó
}

# 2. Criar o objeto GridSearchCV
#   - estimator: nosso modelo base
#   - param_grid: nossa grade de testes
#   - cv=5: validação cruzada com 5 "dobras"
#   - scoring='neg_mean_absolute_error': a métrica que queremos otimizar. Usamos o 'negativo' porque o GridSearch tenta maximizar, e nós queremos minimizar o erro.
#   - n_jobs=-1: usa todos os processadores do seu computador para acelerar o processo.
grid_search = GridSearchCV(
    estimator= RandomForestRegressor(random_state=42),
    param_grid= param_grid,
    cv= 5,
    scoring= 'neg_mean_absolute_error',
    n_jobs= -1,
    verbose= 2 # Mostra o que ele está fazendo em tempo real
)

# 3. Executar a busca
# Esta é a parte que vai demorar. Ele vai treinar 12 combinações * 5 vezes = 60 modelos!
print("Iniciando a busca pelos melhores hiperparâmetros... (Isso pode levar alguns minutos)")
grid_search.fit(X_train, y_train)

# 4. Exibir os melhores resultados
print("\nBusca finalizada!")
print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)

# O resultado do best_score_ virá negativo, basta pegar o valor absoluto
best_mae = -grid_search.best_score_
print(f"\nMelhor MAE (Erro Médio Absoluto) na validação cruzada: € {best_mae:,.2f}")


Iniciando a busca pelos melhores hiperparâmetros... (Isso pode levar alguns minutos)
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Busca finalizada!
Melhores hiperparâmetros encontrados:
{'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}

Melhor MAE (Erro Médio Absoluto) na validação cruzada: € 29,196.69


## Etapa 7: Treinando e Avaliando o Modelo Final Otimizado

In [88]:
# --- Etapa 7: Treinando e Avaliando o Modelo Final Otimizado ---
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# 1. Criar o modelo final com os melhores parâmetros encontrados
print("Treinando o modelo final com os melhores parâmetros...")
final_model = RandomForestRegressor(
    n_estimators= 100,
    max_depth= 10,
    min_samples_split= 2,
    random_state= 42,
    n_jobs=-1 # Usa todos os processadores para o treino final ser mais rápido
)

# 2. Treinar o modelo final com TODOS os dados de treino
final_model.fit(X_train, y_train)
print("Modelo final treinado com sucesso! 🏆")

# 3. Avaliar no conjunto de teste (que o modelo nunca viu antes)
final_y_pred = final_model.predict(X_test)

final_mae = mean_absolute_error(y_test, final_y_pred)
final_r2 = r2_score(y_test, final_y_pred)

# 4. Exibir os resultados finais do projeto!
print("\n--- Avaliação Final do Modelo Otimizado (nos dados de teste) ---")
print(f"Erro Médio Absoluto (MAE) Final: € {final_mae:,.2f}")
print(f"Coeficiente de Determinação (R²) Final: {final_r2:.2%}")

Treinando o modelo final com os melhores parâmetros...
Modelo final treinado com sucesso! 🏆

--- Avaliação Final do Modelo Otimizado (nos dados de teste) ---
Erro Médio Absoluto (MAE) Final: € 28,818.08
Coeficiente de Determinação (R²) Final: 62.37%


In [92]:
# --- CÉLULA FINAL: Exportando os dados para o Power BI ---

# Vamos usar a variável 'df' que contém nosso DataFrame final e limpo.
try:
    output_csv_path = 'dados_para_powerbi.csv'
    
    df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    
    print(f"Dados exportados com sucesso para '{output_csv_path}'! ✅")
    print("Este arquivo está pronto para ser carregado no Power BI.")

except NameError:
    print("Erro: A variável 'df' não foi encontrada. Certifique-se de que a primeira célula do notebook foi executada.")
except Exception as e:
    print(f"Ocorreu um erro ao exportar: {e}")

Dados exportados com sucesso para 'dados_para_powerbi.csv'! ✅
Este arquivo está pronto para ser carregado no Power BI.
